In [18]:
import paramiko, os, sys, argparse, re 
from getpass import getpass
from os.path import exists

# Create the password prompt
thePass = getpass(prompt="SSH Password: ")

# Host info
host = "192.168.6.71"
port = 2222
username = "benjamin.gifford"
password = thePass

try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authentication failed")

In [19]:
sftp = ssh.open_sftp()

# The directory /usr could contain millions of files on a production system so restricting the directories is ideal for the initial review.
command = 'sudo /home/ubuntu/kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin'

#Get output from command
stdin, stdout, stderrr = ssh.exec_command(command)

#Get results from stdoutput
lines = stdout.readlines()
#print(lines)

# Convert list to string
output = ''.join(lines)

# Header output
sepHeader = '' + '### BEGIN ' + command + ' ###\n\n'

# Footer output
sepFooter = '' + '### END ' + command + ' ###\n\n'

cmd_output = sepHeader + output + sepFooter

# Concatonate the header, output, and footer
cmd_output =  sepHeader + output +  sepFooter
#print(cmd_output)

# Save cmd output to a file
with open('results.txt', 'w') as f:
    f.write(cmd_output)
#print(cmd_output)

sftp.close()

In [20]:
sftp = ssh.open_sftp()

# Upload results.txt
uploadFiles = ['results.txt']

# Run lsof on remote system          
# command = "sudo -S lsof -i -n | awk '{print $2}' "
command = 'sudo -S lsof -i -n'

# Get the output from the command
stdin, stdout, stderr  = ssh.exec_command(command)

# Input sudo password
stdin.write(thePass + "\n")

# Get results  from stdout
lines =  stdout.readlines()

# Convert list to a string
output =  ''.join(lines)

# Header output
sepHeader = '' + '### BEGIN ' + command + ' ###\n\n'

# Footer output
sepFooter = '' + '### END ' + command + ' ###\n\n'

# Concatonate the header, output, and footer
cmd_output =  sepHeader + output +  sepFooter

# Save cmd output to a file
with open('results2.txt', 'w') as f:
    f.write(cmd_output)

# End SFTP Session
ssh.close()
sftp.close()

In [21]:
# Search for items in list 1 within list 2
list1 = ['519423', '561559']
list2 = ['593','603','603','830','855','855','935','935','561559','600020', '600170', '600390', '600489', '600523', '600626', '600716', '600818', '600854', '600947', '600952', '601052', '601067', '601169', '601242', '601357', '601383', '601485', '601534', '601633', '601653', '601752', '601759', '601861', '601881', '601977', '601982', '602084', '602108', '602204', '602220', '602318', '602330', '602429', '602470', '602568', '602604', '602705', '602900', '603028', '603036', '603134', '603284', '603390']

results = ({i:j  for j in list1 for i in list2 if re.match(i,j)})
print(results)

matchPID = '561559'

with open ('results2.txt', 'r') as f:
    for line in f:
        y = re.findall(r''+matchPID+'', line)
        print (line)
 

{'561559': '561559'}
### BEGIN sudo -S lsof -i -n ###



COMMAND      PID             USER   FD   TYPE  DEVICE SIZE/OFF NODE NAME

systemd-n    593  systemd-network   19u  IPv4   19178      0t0  UDP 10.223.0.144:bootpc 

systemd-r    603  systemd-resolve   12u  IPv4   19238      0t0  UDP 127.0.0.53:domain 

systemd-r    603  systemd-resolve   13u  IPv4   19239      0t0  TCP 127.0.0.53:domain (LISTEN)

lwsmd        855             root   39u  IPv4 2803377      0t0  TCP 10.223.0.144:56702->192.168.4.4:ldap (ESTABLISHED)

sshd         935             root    3u  IPv4   24209      0t0  TCP *:ssh (LISTEN)

sshd         935             root    4u  IPv6   24221      0t0  TCP *:ssh (LISTEN)

egrep     561559         p.dalton    5u  IPv6 2257680      0t0  TCP *:http-alt (LISTEN)

sshd      600020             root    4u  IPv4 2507834      0t0  TCP 10.223.0.144:ssh->192.168.4.33:52599 (ESTABLISHED)

sshd      600170    dylan.navarro    4u  IPv4 2507834      0t0  TCP 10.223.0.144:ssh->192.168.4.33